In [ ]:
import locale
locale.getpreferredencoding = lambda: 'UTF-8'

%env LC_ALL=C.UTF-8
%env LANG=C.UTF-8

!pip install roboflow
!pip install ultralytics
!pip install matplotlib
import os
import yaml
import json
from sklearn.model_selection import train_test_split
from google.colab import drive
import shutil

# 挂载 Google Drive
drive.mount('/content/drive')

from ultralytics import YOLO

# 定义模型
model = YOLO('yolov8n.pt')  # 请确保 'yolov8n.pt' 文件存在，或使用您所需的模型文件

# 设置图像文件夹路径
image_folder = '/content/drive/MyDrive/p/training/images'
label_file = '/content/drive/MyDrive/p/training/label.json'  # 标签文件路径

# 获取所有图像文件路径
all_images = [os.path.join(image_folder, img) for img in os.listdir(image_folder) if img.endswith('.png') or img.endswith('.jpg')]

# 将数据集拆分为 70% 训练，15% 验证，15% 测试
train_images, temp_images = train_test_split(all_images, test_size=0.3, random_state=42)
val_images, test_images = train_test_split(temp_images, test_size=0.5, random_state=42)

# 创建新的文件夹结构
base_path = '/content/drive/MyDrive/p'
train_folder = os.path.join(base_path, 'train_split/images')
val_folder = os.path.join(base_path, 'val_split/images')
test_folder = os.path.join(base_path, 'test_split/images')
train_labels_folder = os.path.join(base_path, 'train_split/labels')
val_labels_folder = os.path.join(base_path, 'val_split/labels')
test_labels_folder = os.path.join(base_path, 'test_split/labels')

os.makedirs(train_folder, exist_ok=True)
os.makedirs(val_folder, exist_ok=True)
os.makedirs(test_folder, exist_ok=True)
os.makedirs(train_labels_folder, exist_ok=True)
os.makedirs(val_labels_folder, exist_ok=True)
os.makedirs(test_labels_folder, exist_ok=True)

# 将图像文件复制到相应的文件夹
def move_files(file_list, dest_folder):
    for file_path in file_list:
        shutil.copy(file_path, os.path.join(dest_folder, os.path.basename(file_path)))

move_files(train_images, train_folder)
move_files(val_images, val_folder)
move_files(test_images, test_folder)

# 将 label.json 转换为 YOLO 格式的单独标签文件
with open(label_file, 'r') as f:
    data = json.load(f)

# 为每张图像生成 YOLO 格式标签文件
def create_yolo_labels(images, labels_folder):
    for image_data in data['images']:
        image_id = image_data['id']
        file_name = image_data['file_name']
        if os.path.join(image_folder, file_name) in images:
            label_file = os.path.join(labels_folder, file_name.replace('.png', '.txt').replace('.jpg', '.txt'))

            annotations = [ann for ann in data['annotations'] if ann['image_id'] == image_id]

            with open(label_file, 'w') as lf:
                for ann in annotations:
                    category_id = ann['category_id_3']  # 使用 category_id_1
                    bbox = ann['bbox']

                    # 计算 YOLO 格式的 x_center, y_center, width, height（归一化）
                    x_center = (bbox[0] + bbox[2] / 2) / image_data['width']
                    y_center = (bbox[1] + bbox[3] / 2) / image_data['height']
                    width = bbox[2] / image_data['width']
                    height = bbox[3] / image_data['height']

                    # 写入一行数据
                    lf.write(f"{category_id} {x_center} {y_center} {width} {height}\n")


# 为训练、验证和测试集生成标签文件
create_yolo_labels(train_images, train_labels_folder)
create_yolo_labels(val_images, val_labels_folder)
create_yolo_labels(test_images, test_labels_folder)

# 创建 data.yaml 文件内容
data_config = {
    'train': os.path.join(base_path, 'train_split/images'),
    'val': os.path.join(base_path, 'val_split/images'),
    'test': os.path.join(base_path, 'test_split/images'),
    'nc': 4,  # 假设只有一个类别，按实际情况修改
    'names': ['Caries','Deep Caries','Periapical Lesion','Impacted']  # 假设类别名称为 "tooth"，按实际情况修改
}

# 将配置写入 data.yaml 文件
config_path = os.path.join(base_path, 'data.yaml')
with open(config_path, 'w', encoding='utf-8') as f:
    yaml.dump(data_config, f)

print("data.yaml 文件已生成。")

# 模型训练代码
#model.train(data=config_path, epochs=5)
model.train(data=config_path, epochs=5, project='/content/drive/MyDrive/p', name='11_9pt')



import matplotlib.pyplot as plt
from ultralytics import YOLO
from PIL import Image

# 定义测试集的配置文件路径（data.yaml 文件路径）
config_path = '/content/drive/MyDrive/p/data.yaml'  # 替换为实际路径

# 加载训练好的最佳模型权重文件
trained_model = YOLO('/content/drive/MyDrive/p/11_9pt/weights/best.pt')  # 替换为训练输出的最佳权重路径

# 使用测试集进行评估
results = trained_model.val(data=config_path, split='test')  # `split='test'` 表示使用 data.yaml 中定义的测试集



# 提取评估指标
precision = results.box.p.mean()    # 平均精度（Precision）
recall = results.box.r.mean()       # 平均召回率（Recall）
map50 = results.box.map50           # 平均AP（AP@0.5）
map50_95 = results.box.map           # 平均AP（AP@0.5:0.95）
f1_score = results.box.f1.mean()    # 平均F1分数

print(f"Mean Precision: {precision:.4f}")
print(f"Mean Recall: {recall:.4f}")
print(f"Mean AP@0.5: {map50:.4f}")
print(f"Mean AP@0.5-0.95: {map50_95:.4f}")
print(f"F1 Score: {f1_score:.4f}")




# 可视化评估指标
metrics = ['Precision', 'Recall', 'AP@0.5', 'AP@0.5:0.95']
scores = [precision, recall, map50, map50_95]

plt.figure(figsize=(8, 6))
plt.bar(metrics, scores, color='skyblue')
plt.ylim(0, 1)
plt.xlabel('Metrics')
plt.ylabel('Score')
plt.title('Model Evaluation Metrics on Test Set')
plt.show()

# # 对单张测试图片进行预测示例
# test_images_folder = '/content/drive/MyDrive/p/test_split/images'  # 替换为测试图片文件夹路径
# test_image_path = f"{test_images_folder}/example_image.jpg"  # 替换为具体测试图片路径
# predictions = trained_model.predict(source=test_image_path, save=True, project='/content/drive/MyDrive/p', name='test_predictions')

# # 显示预测后的图片
# output_image_path = '/content/drive/MyDrive/p/test_predictions/example_image.jpg'  # 替换为预测输出图片路径
# output_image = Image.open(output_image_path)
# output_image.show()


from glob import glob
from PIL import Image

test_images_folder = '/content/drive/MyDrive/p/test_split/images'  # 替换为您的测试图片文件夹路径
image_paths = glob(f"{test_images_folder}/*.jpg")  # 获取所有jpg文件路径

for image_path in image_paths:
    predictions = trained_model.predict(source=image_path, save=True, project='/content/drive/MyDrive/p', name='test_predictions')
    output_image = Image.open(image_path)
    output_image.show()


